### Ansible

```
Using Ansible base steps for the future data hub environment.

Install on mac:
brew update
brew install ansible
(or us pip to install ansible module using python alternatively)

test:
ansible --version

Use git as version system:
- use master as dev branch (to prevent accidental pushes to prod) 
- create prod branch (or test etc)
- USe branches for long term changes (feature branch)
- Always deploy right after pushing to production.
- Use small changes, not 1 big change.

```

### Testing Ansible without cloud provider

```
Well to prevent local gluther just create a server in aws manual
We create an ec nano server, with port 22 and 33434 (ping) open to the your public ip. Also we open port 80 for a webserver.
Download the key text put this in a <server>.pem file ($HOME/.ssh/<server>.pem)
Check out the public ip in the aws console: 

Go to the ec2 server
cd /Users/michelnossin/.ssh
chmod 400 *.pem
ssh -i michel-ansible-test.pem ec2-user@<public ip>
exit

#Now add the key to ssh so we dont need to specify
ssh-add michel-ansible-test.pem
ssh ec2-user@<public ip>

```



```
Some ansible tests:
ansible --help
<check out ansi configdir near inventory part!>
ansible all -i 35.157.62.157, -m ping -u ec2-user
ansible all -i 35.157.62.157, -m shell -a "echo 'hallo'" -u ec2-user

-m is for modules, with optional -a as arguments

```

### Playbooks

example, will put in current repository.
execute with: 
ansible-playbook -i 35.157.62.157, http-server.yaml

``` 
---
- hosts: all
  remote_user: ec2-user
  become: True
  tasks:
  - name: Ensure the HTTPd package is installed
    yum:
      name: httpd
      state: present
  - name: Ensure the HTTPd service is enabled and running
    service:
      name: httpd
      state: started
      enabled: True

```    
        
 Running multiple times gives changed values instead of ok, this means idempotency. Monitor change values therefor.
 debug: run with -v , or multiple like -vvv to get extra info

Let's add in same yml file set and get of variables, to be used while
reusing yaml playbooks.
```
variables:

tasks:
- name: Set variable 'x' 
  set_fact:
    x: 'michel'
- name: Get variable 'x'
  debug:
    msg: '{{ x }}'
``` 

Or pass -e 'x=michel' at the prompt, so set is not needed. Only issue, you lose that configuration, its not stored

Optionally add this to make sure our webserver can pass firewall. We will not add this as our webserver allready works.
```
- name: Ensure HTTP can pass the firewall
       firewalld:
         service: http
         state: enabled
         permanent: True
         immediate: True
       become: True
     - name: Ensure HTTPS can pass the firewall
       firewalld:
         service: https
         state: enabled
         permanent: True
         immediate: True
become: True
```

Lets publish our page:
```
- name: Ensure the website is present and updated
       template:
         src: index.html.j2
         dest: /var/www/html/index.html
         owner: root
         group: root
         mode: 0644

We will put this in tje j2 file

<html>
       <body>
           <h1>Hello Michel</h1>
       </body>
</html>

```

go to http://35.157.62.157 , it works